In [11]:
import numpy as np
import pandas as pd

In [12]:
import pandas as pd

file_path = './Kaggle/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'
df = pd.read_csv(file_path)

# Display first few records
print(df.head())

# Check the dataset info
print(df.info())


                     id             dateAdded           dateUpdated  \
0  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
1  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
2  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
3  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
4  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   

                                                name       asins   brand  \
0  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
1  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
2  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
3  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
4  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   

                                          categories primaryCategories  \
0  Computers,Electronics Features,Tablets,

In [ ]:
# Handle missing values by filling or dropping columns (optional)
df.loc[:, 'reviews.title'] = df['reviews.title'].fillna('No title')  # Fill missing titles
df.loc[:, 'reviews.dateAdded'] = df['reviews.dateAdded'].fillna('Unknown')  # Fill missing dates

# Convert 'dateAdded' and 'dateUpdated' to datetime
df.loc[:, 'dateAdded'] = pd.to_datetime(df['dateAdded'], errors='coerce')
df.loc[:, 'dateUpdated'] = pd.to_datetime(df['dateUpdated'], errors='coerce')

# Display cleaned data info
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   5000 non-null   object 
 1   dateAdded            5000 non-null   object 
 2   dateUpdated          5000 non-null   object 
 3   name                 5000 non-null   object 
 4   asins                5000 non-null   object 
 5   brand                5000 non-null   object 
 6   categories           5000 non-null   object 
 7   primaryCategories    5000 non-null   object 
 8   imageURLs            5000 non-null   object 
 9   keys                 5000 non-null   object 
 10  manufacturer         5000 non-null   object 
 11  manufacturerNumber   5000 non-null   object 
 12  reviews.date         5000 non-null   object 
 13  reviews.dateAdded    5000 non-null   object 
 14  reviews.dateSeen     5000 non-null   object 
 15  reviews.doRecommend  5000 non-null   b

In [ ]:
import pandas as pd

# Check for duplicate rows
duplicate_count = df_cleaned.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Remove duplicate rows
df_no_duplicates = df_cleaned.drop_duplicates()
print(f"Dataset shape after removing duplicates: {df_no_duplicates.shape}")

text_columns = [col for col in df.columns if 'review' in col.lower() or 'text' in col.lower()]
if text_columns:
    review_col = text_columns[8]
    print(f"\nChecking duplicates in '{review_col}' column")
    duplicate_reviews = df.duplicated(subset=[review_col]).sum()
    print(f"Number of duplicate reviews: {duplicate_reviews}")
    
    # Remove rows with duplicate reviews
    df_unique_reviews = df.drop_duplicates(subset=[review_col])
    print(f"Dataset shape after removing duplicate reviews: {df_unique_reviews.shape}")

# Save the deduplicated dataset
output_path = './Kaggle/Datafiniti_Amazon_Consumer_Reviews_Deduplicated.csv'
df_unique_reviews.to_csv(output_path, index=False)
print(f"Deduplicated dataset saved to {output_path}")

Number of duplicate rows: 0
Dataset shape after removing duplicates: (4385, 24)

Checking duplicates in 'reviews.text' column
Number of duplicate reviews: 615
Dataset shape after removing duplicate reviews: (4385, 24)
Deduplicated dataset saved to ./Kaggle/Datafiniti_Amazon_Consumer_Reviews_Deduplicated.csv


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import classification_report

# Load dataset
df = df_unique_reviews

# Map ratings to sentiment
sentiment_mapping = {1: "negative", 2: "negative", 3: "neutral", 4: "positive", 5: "positive"}
df["sentiment"] = df["reviews.rating"].map(sentiment_mapping)

# Drop missing values
df = df.dropna(subset=["reviews.text", "sentiment"])

# Handle class imbalance
positive = df[df["sentiment"] == "positive"]
neutral = df[df["sentiment"] == "neutral"]
negative = df[df["sentiment"] == "negative"]

# Resample to balance classes
min_samples = min(len(positive), len(neutral), len(negative))
positive_downsampled = resample(positive, replace=False, n_samples=min_samples, random_state=42)
neutral_downsampled = resample(neutral, replace=True, n_samples=min_samples, random_state=42)
negative_upsampled = resample(negative, replace=True, n_samples=min_samples, random_state=42)

# Combine balanced dataset
balanced_df = pd.concat([positive_downsampled, neutral_downsampled, negative_upsampled])

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(balanced_df["reviews.text"], balanced_df["sentiment"], test_size=0.2, random_state=42)

# Text vectorization
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train model
model = RandomForestClassifier(class_weight="balanced", n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

# Evaluate model
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.94      0.75      0.83        20
     neutral       0.48      0.73      0.58        15
    positive       0.83      0.68      0.75        22

    accuracy                           0.72        57
   macro avg       0.75      0.72      0.72        57
weighted avg       0.78      0.72      0.73        57



# Data Clustering

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Load data
df = pd.read_csv("./Kaggle/Datafiniti_Amazon_Consumer_Reviews_Deduplicated.csv")
df = df.dropna(subset=["name", "reviews.text"])

# Preprocess text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation/numbers
    return text

df["cleaned_text"] = df["name"] + " " + df["reviews.text"]
df["cleaned_text"] = df["cleaned_text"].apply(clean_text)

# Vectorize text
vectorizer = TfidfVectorizer(stop_words="english", max_features=8000)
X_tfidf = vectorizer.fit_transform(df["cleaned_text"])

# Apply K-Means clustering
num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(X_tfidf)

# Evaluate clustering quality
print("\n--- Clustering Evaluation Metrics ---")

# 1. Silhouette Score (higher is better, range: -1 to 1)
silhouette = silhouette_score(X_tfidf, df["cluster"])
print(f"Silhouette Score: {silhouette:.4f}")

# 2. Davies-Bouldin Index (lower is better)
davies_bouldin = davies_bouldin_score(X_tfidf.toarray(), df["cluster"])
print(f"Davies-Bouldin Index: {davies_bouldin:.4f}")

# 3. Calinski-Harabasz Index (higher is better)
calinski_harabasz = calinski_harabasz_score(X_tfidf.toarray(), df["cluster"])
print(f"Calinski-Harabasz Index: {calinski_harabasz:.4f}")

# 4. Cluster size distribution
cluster_sizes = Counter(df["cluster"])
print("\nCluster Size Distribution:")
for cluster, size in sorted(cluster_sizes.items()):
    print(f"Cluster {cluster}: {size} items ({size/len(df)*100:.2f}%)")

# 5. Find top terms per cluster
def get_top_terms_per_cluster(vectorizer, kmeans, n_terms=10):
    terms = vectorizer.get_feature_names_out()
    cluster_terms = {}
    
    for i in range(kmeans.n_clusters):
        # Get indices of documents in this cluster
        cluster_docs = df[df["cluster"] == i].index
        if len(cluster_docs) == 0:
            continue
            
        cluster_tfidf = X_tfidf[cluster_docs]
        cluster_vec = np.array(cluster_tfidf.sum(axis=0)).flatten()

        top_indices = cluster_vec.argsort()[-n_terms:][::-1]

        top_terms = [terms[idx] for idx in top_indices]
        cluster_terms[i] = top_terms
    
    return cluster_terms

top_terms = get_top_terms_per_cluster(vectorizer, kmeans)
print("\nTop Terms Per Cluster:")
for cluster, terms in top_terms.items():
    print(f"Cluster {cluster}: {', '.join(terms)}")

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_tfidf.toarray())

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df["cluster"], cmap='viridis', alpha=0.5)
plt.colorbar(scatter, label='Cluster')
plt.title('PCA Visualization of Clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.savefig('./Kaggle/cluster_visualization.png')
plt.close()

inertia_values = []
silhouette_values = []
k_range = range(2, 11)

for k in k_range:
    kmeans_test = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans_test.fit_predict(X_tfidf)
    inertia_values.append(kmeans_test.inertia_)
    silhouette_values.append(silhouette_score(X_tfidf, cluster_labels))

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(k_range, inertia_values, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')

plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_values, 'ro-')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')
plt.tight_layout()
plt.savefig('./Kaggle/optimal_clusters.png')
plt.close()

# Map clusters to categories manually
cluster_map = {
    0: "Ebook Readers",
    1: "Batteries",
    2: "Accessories",
    3: "Non-Electronics",
    4: "Cables & Adapters",
    5: "Smart Home Devices"
}
df["category"] = df["cluster"].map(cluster_map)

# 8. Evaluate category coherence
print("\nCategory Coherence:")
for cluster, category in cluster_map.items():
    cluster_df = df[df["cluster"] == cluster]
    if len(cluster_df) > 0:
        print(f"\nCategory: {category} (Cluster {cluster})")
        print(f"Number of items: {len(cluster_df)}")
        print(f"Top terms: {', '.join(top_terms[cluster][:5])}")
        print(f"Sample product names: {', '.join(cluster_df['name'].sample(min(3, len(cluster_df))).tolist())}")

# Save and preview results
df.to_csv("./Kaggle/clustered_reviews.csv", index=False)
print("\nClustering complete. Results saved to ./Kaggle/clustered_reviews.csv")
print(df[["name", "category"]].head(10))

c:\Users\rfria\Projects\ironhacks\customer-reviews\.env\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\rfria\Projects\ironhacks\customer-reviews\.env\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\rfria\scoop\apps\python\current\Lib\subprocess.py", line 556, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rfria\scoop\apps\python\current\Lib\subprocess.py", line 1038, in __init__
    self._ex


--- Clustering Evaluation Metrics ---
Silhouette Score: 0.0538
Davies-Bouldin Index: 4.3277
Calinski-Harabasz Index: 115.4831

Cluster Size Distribution:
Cluster 0: 882 items (20.11%)
Cluster 1: 691 items (15.76%)
Cluster 2: 976 items (22.26%)
Cluster 3: 267 items (6.09%)
Cluster 4: 979 items (22.33%)
Cluster 5: 590 items (13.45%)

Top Terms Per Cluster:
Cluster 0: black, kindle, offers, special, tablet, includes, wifi, display, gb, ereader
Cluster 1: kids, case, edition, kidproof, tablet, blue, gb, display, wifi, green
Cluster 2: speaker, bluetooth, alexaenabled, echo, screen, amazon, tap, portable, love, great
Cluster 3: ips, brand, new, gb, kindle, blue, tablet, amazon, display, wifi
Cluster 4: hd, allnew, tablet, magenta, includes, special, offers, gb, display, wifi
Cluster 5: plus, hub, builtin, silver, echo, amazon, alexa, love, great, lights

Category Coherence:

Category: Ebook Readers (Cluster 0)
Number of items: 882
Top terms: black, kindle, offers, special, tablet
Sample pr

# blog post like article

In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import gc
import time

df = pd.read_csv("/content/clustered_reviews.csv")

df = df.dropna(subset=["reviews.text", "reviews.rating", "name", "category"])

print("Loading model for article generation...")
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with memory optimizations
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Enable gradient checkpointing to save memory
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()

# Function to generate article with memory management
def generate_article(prompt, max_length=800):
    # Clear CUDA cache if using GPU
    if device.type == "cuda":
        torch.cuda.empty_cache()
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)

        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            num_beams=4,
            early_stopping=True,
            temperature=0.7,
            no_repeat_ngram_size=3,
            do_sample=True,
            length_penalty=1.0
        )
        
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Clean up to free memory
        del inputs, outputs
        if device.type == "cuda":
            torch.cuda.empty_cache()
        
        return result
    
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print("  CUDA out of memory, trying with smaller context...")
            inputs = tokenizer(prompt[:len(prompt)//2], return_tensors="pt", max_length=512, truncation=True).to(device)
            outputs = model.generate(
                inputs["input_ids"],
                max_length=max_length//2,
                num_beams=2,
                early_stopping=True
            )
            return tokenizer.decode(outputs[0], skip_special_tokens=True)
        else:
            raise e

# Process categories in smaller batches to manage memory
articles = []
unique_categories = df["category"].unique()
print(f"Found {len(unique_categories)} unique categories")

# Process categories in batches
batch_size = 2
for i in range(0, len(unique_categories), batch_size):
    batch_categories = unique_categories[i:i+batch_size]
    print(f"Processing batch {i//batch_size + 1}/{(len(unique_categories) + batch_size - 1)//batch_size}")
    
    for category in batch_categories:
        print(f"Processing category: {category}")
        cat_df = df[df["category"] == category]
        
        # Skip if category has no products
        if len(cat_df) == 0:
            print(f"  No products found in category {category}, skipping...")
            continue
        
        # Identify top 3 and worst product
        product_ratings = cat_df.groupby("name")["reviews.rating"].mean().sort_values()
        
        # Skip if no products with ratings
        if len(product_ratings) == 0:
            print(f"  No products with ratings in category {category}, skipping...")
            continue
        
        worst_product = product_ratings.index[0]
        
        # Get top products (up to 3)
        if len(product_ratings) >= 3:
            top_products = product_ratings.index[-3:].tolist()
        else:
            top_products = product_ratings.index.tolist()
        
        # Extract key positive points for top products (more memory efficient)
        positive_points = {}
        for product in top_products:
            product_df = cat_df[cat_df["name"] == product]
            positive_reviews = product_df[product_df["reviews.rating"] >= 4]["reviews.text"]
            if len(positive_reviews) > 0:
                # Take fewer samples to reduce memory usage
                sample_size = min(3, len(positive_reviews))
                positive_text = " ".join(positive_reviews.sample(sample_size).tolist())
                positive_points[product] = positive_text[:200]  # Shorter text
        
        # Extract key negative points for worst product
        negative_points = ""
        worst_product_df = cat_df[cat_df["name"] == worst_product]
        negative_reviews = worst_product_df[worst_product_df["reviews.rating"] <= 2]["reviews.text"]
        if len(negative_reviews) > 0:
            sample_size = min(2, len(negative_reviews))
            negative_points = " ".join(negative_reviews.sample(sample_size).tolist())[:200]
        
        # Create prompt for article generation (simplified)
        article_prompt = f"""
        Write a detailed blog post about Amazon products in the category '{category}'.
        
        FORMAT:
        - Title: [Catchy title]
        - Introduction
        - Best Products: {', '.join(top_products)}
        - Worst Product: {worst_product}
        - Conclusion
        
        Top product features: {str(positive_points)[:500]}
        Issues with worst product: {negative_points}
        
        Make the article informative and helpful for consumers.
        """
        
        # Generate article
        print(f"  Generating article...")
        article = generate_article(article_prompt, max_length=800)
        
        articles.append({
            "category": category,
            "top_products": top_products,
            "worst_product": worst_product,
            "article": article
        })
        print(f"  Article generated: {len(article)} characters")
        
        # Save progress after each category to avoid losing work
        temp_df = pd.DataFrame(articles)
        temp_df.to_csv(f"/content/category_blog_articles_progress.csv", index=False)
        
        # Force garbage collection
        gc.collect()
        if device.type == "cuda":
            torch.cuda.empty_cache()
        
        # Add a small delay to let system recover
        time.sleep(2)
    
    # Clear memory between batches
    gc.collect()
    if device.type == "cuda":
        torch.cuda.empty_cache()

# Convert to DataFrame and save
articles_df = pd.DataFrame(articles)
articles_df.to_csv("/content/category_blog_articles.csv", index=False)

print("\nArticle generation complete!")
print(f"Generated {len(articles)} articles")
print("Articles saved to /content/category_blog_articles.csv")

# Preview a sample article
if len(articles) > 0:
    sample_idx = 0
    print("\nSample Article Preview:")
    print(f"Category: {articles[sample_idx]['category']}")
    print(f"Top Products: {', '.join(articles[sample_idx]['top_products'])}")
    print(f"Worst Product: {articles[sample_idx]['worst_product']}")
    print("\nArticle:")
    print(articles[sample_idx]['article'])

c:\Users\rfria\Projects\ironhacks\customer-reviews\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model for article generation...


c:\Users\rfria\Projects\ironhacks\customer-reviews\.env\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rfria\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Using device: cpu
Generating articles for 6 categories...
Processing category: Ebook Readers
  Generating article...


c:\Users\rfria\Projects\ironhacks\customer-reviews\.env\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


  Article generated: 128 characters
Processing category: Accessories
  Generating article...


c:\Users\rfria\Projects\ironhacks\customer-reviews\.env\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 